In [1]:
import polars as pl
import gzip

In [2]:
df = pl.read_csv('~/buckets/b1/datasets/competencia_02.csv.gz', infer_schema_length=100000, dtypes={'foto_mes':str})

In [3]:
query = """
    select foto_mes, numero_de_cliente, mcomisiones, lag(mcomisiones,1) over (partition by numero_de_cliente order by foto_mes) as mcomisiones_1 
    from frame where numero_de_cliente=29183981;    
"""

#df_client = df.filter(pl.col('numero_de_cliente')==29183981).sort('foto_mes')
#df_client.select([pl.col('foto_mes'), pl.col('numero_de_cliente'), pl.col('mcomisiones'), pl.col('mcomisiones_lag_1'), pl.col('mcomisiones_lag_2'), pl.col('mcomisiones_lag_3')])
columns = df.columns 
columns.remove('foto_mes')
columns.remove('numero_de_cliente')
columns.remove('clase_ternaria')

In [8]:
df = df.sort('foto_mes')

In [9]:
max_lag = 3
for column in columns:
    for lag in range(1, max_lag+1):    
        column_lag = column + '_lag_' + str(lag)
        df = df.with_columns([pl.col(column).shift(lag).over('numero_de_cliente').alias(column_lag)])

In [11]:
# Save the result as gzip
with gzip.open('~/buckets/b1/datasets/competencia_02_3lags.csv.gz', 'wb') as f:
    df.write_csv(f)
